In [365]:
import json
import pandas as pd
from datetime import datetime
from listas import priorfalls_list, delirium_list, dementia_list, depresyndr_list, uriincont_list, fecincont_list, pressulc_list, osteopor_list, sarcopenia_list, sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list, ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list, psicosis_list

from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

taula = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


In [366]:
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...


In [367]:
AMB_PA = taula[taula["PA_diagnosticada"] == 1.0]
AMB_PA_MECVV = taula[
    (taula["P_diagnosticada"] == 1.0) & (taula["Dias_totales_ingresado"] < 30) & (taula["Disfagia_mecvvs"] == 1.0) & (taula["alteracioEficacia_mecvvs"] == 1.0) |
    (taula["P_diagnosticada"] == 1.0) & (taula["Dias_totales_ingresado"] < 30) & (taula["Disfagia_mecvvs"] == 1.0) & (taula["alteracioSeguretat_mecvvs"] == 1.0)]   
SENSE_PA = taula[
          (taula["P_diagnosticada"] == 1.0) |
          (taula["P_diagnosticada"] == 1.0) & (taula["Disfagia_mecvvs"] == 1.0) & (taula["alteracioEficacia_mecvvs"] == 1.0) & (taula["Dias_totales_ingresado"] > 30) |
          (taula["P_diagnosticada"] == 1.0) & (taula["Disfagia_mecvvs"] == 1.0) & (taula["alteracioSeguretat_mecvvs"] == 1.0) & (taula["Dias_totales_ingresado"] > 30)]

# Síndromes geriàtrics

## Index de Charslton

## Caigudes prèvies

In [368]:
caprev_AMB_PA= (AMB_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA:.2f}%")

Percentatge de caigudes prèvies: 5.88%


In [369]:
caprev_AMB_PA_MECVV= (AMB_PA_MECVV['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA_MECVV:.2f}%")

Percentatge de caigudes prèvies: 0.00%


In [370]:
caprev_SENSE_PA= (SENSE_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_SENSE_PA:.2f}%")

Percentatge de caigudes prèvies: 6.12%


#### Pvalor

##### Normalitat

In [371]:
stat, p_value = shapiro(AMB_PA['caidas_previas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [372]:
stat, p_value = shapiro(AMB_PA_MECVV['caidas_previas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [373]:
stat, p_value = shapiro(SENSE_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [374]:
stat, p_value_12 = mannwhitneyu(AMB_PA['caidas_previas'], AMB_PA_MECVV['caidas_previas'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.49002535950179893


In [375]:
stat, p_value_13 = mannwhitneyu(AMB_PA['caidas_previas'], SENSE_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.9858477456716862


In [376]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['caidas_previas'], SENSE_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.44143809205435813


## Deliris

In [377]:
del_AMB_PA= (AMB_PA['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA:.2f}%")

Percentatge de deliris: 23.53%


In [378]:
del_AMB_PA_MECVV= (AMB_PA_MECVV['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA_MECVV:.2f}%")

Percentatge de deliris: 40.00%


In [379]:
del_SENSE_PA= (SENSE_PA['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_SENSE_PA:.2f}%")

Percentatge de deliris: 18.37%


#### Pvalor

##### Normalitat

In [380]:
stat, p_value = shapiro(AMB_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [381]:
stat, p_value = shapiro(AMB_PA_MECVV['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6405
Valor p = 0.0002
El nombre de files no segueix una distribució normal.


In [382]:
stat, p_value = shapiro(SENSE_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [383]:
stat, p_value_12 = mannwhitneyu(AMB_PA['delirios'], AMB_PA_MECVV['delirios'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.39174655228054167


In [384]:
stat, p_value_13 = mannwhitneyu(AMB_PA['delirios'], SENSE_PA['delirios'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.6549000779898075


In [385]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['delirios'], SENSE_PA['delirios'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.13960802912048415


## Demència

In [386]:
dem_AMB_PA= (AMB_PA['demencia'] == 1.0).mean() * 100
print(f"Percentatge de demencia: {dem_AMB_PA:.2f}%")

Percentatge de demencia: 47.06%


In [387]:
dem_AMB_PA_MECVV= (AMB_PA_MECVV['demencia'] == 1.0).mean() * 100
print(f"Percentatge de demencia: {dem_AMB_PA_MECVV:.2f}%")

Percentatge de demencia: 40.00%


In [388]:
dem_SENSE_PA= (SENSE_PA['demencia'] == 1.0).mean() * 100
print(f"Percentatge de demencia: {dem_SENSE_PA:.2f}%")

Percentatge de demencia: 30.61%


#### Pvalor

##### Normalitat

In [389]:
stat, p_value = shapiro(AMB_PA['demencia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6418
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [390]:
stat, p_value = shapiro(AMB_PA_MECVV['demencia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6405
Valor p = 0.0002
El nombre de files no segueix una distribució normal.


In [391]:
stat, p_value = shapiro(SENSE_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [392]:
stat, p_value_12 = mannwhitneyu(AMB_PA['demencia'], AMB_PA_MECVV['demencia'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.7484826918143946


In [393]:
stat, p_value_13 = mannwhitneyu(AMB_PA['demencia'], SENSE_PA['demencia'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.22703869678020527


In [394]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['demencia'], SENSE_PA['demencia'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.5743933608939569


## Síndrome depressiu

In [395]:
depre_AMB_PA= (AMB_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA:.2f}%")

Percentatge de síndrome depressiu: 23.53%


In [396]:
depre_AMB_PA_MECVV= (AMB_PA_MECVV['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA_MECVV:.2f}%")

Percentatge de síndrome depressiu: 0.00%


In [397]:
depre_SENSE_PA= (SENSE_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_SENSE_PA:.2f}%")

Percentatge de síndrome depressiu: 14.29%


#### Pvalor

##### Normalitat

In [398]:
stat, p_value = shapiro(AMB_PA['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [399]:
stat, p_value = shapiro(AMB_PA_MECVV['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [400]:
stat, p_value = shapiro(SENSE_PA['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4168
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [401]:
stat, p_value_12 = mannwhitneyu(AMB_PA['sindrome_depresivo'], AMB_PA_MECVV['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.11180527770715322


In [402]:
stat, p_value_13 = mannwhitneyu(AMB_PA['sindrome_depresivo'], SENSE_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.3880726918610017


In [403]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['sindrome_depresivo'], SENSE_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.21340049774345338


## Incontinència urinària

In [404]:
ic_AMB_PA= (AMB_PA['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_AMB_PA:.2f}%")

Percentatge de incontinència urinària: 29.41%


In [405]:
ic_AMB_PA_MECVV= (AMB_PA_MECVV['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_AMB_PA_MECVV:.2f}%")

Percentatge de incontinència urinària: 30.00%


In [406]:
ic_SENSE_PA= (SENSE_PA['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_SENSE_PA:.2f}%")

Percentatge de incontinència urinària: 14.29%


#### Pvalor

##### Normalitat

In [407]:
stat, p_value = shapiro(AMB_PA['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [408]:
stat, p_value = shapiro(AMB_PA_MECVV['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5942
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [409]:
stat, p_value = shapiro(SENSE_PA['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4168
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [410]:
stat, p_value_12 = mannwhitneyu(AMB_PA['incont_uri'], AMB_PA_MECVV['incont_uri'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 1.0


In [411]:
stat, p_value_13 = mannwhitneyu(AMB_PA['incont_uri'], SENSE_PA['incont_uri'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.17015925929193942


In [412]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['incont_uri'], SENSE_PA['incont_uri'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.237514563226218


## Incontinència fecal 

In [413]:
ifec_AMB_PA= (AMB_PA['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_AMB_PA:.2f}%")

Percentatge de incontinència fecal: 29.41%


In [414]:
ifec_AMB_PA_MECVV= (AMB_PA_MECVV['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_AMB_PA_MECVV:.2f}%")

Percentatge de incontinència fecal: 10.00%


In [415]:
ifec_SENSE_PA= (SENSE_PA['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_SENSE_PA:.2f}%")

Percentatge de incontinència fecal: 12.24%


#### Pvalor

##### Normalitat

In [416]:
stat, p_value = shapiro(AMB_PA['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [417]:
stat, p_value = shapiro(AMB_PA_MECVV['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [418]:
stat, p_value = shapiro(SENSE_PA['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3844
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [419]:
stat, p_value_12 = mannwhitneyu(AMB_PA['incont_fec'], AMB_PA_MECVV['incont_fec'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.2649031747093591


In [420]:
stat, p_value_13 = mannwhitneyu(AMB_PA['incont_fec'], SENSE_PA['incont_fec'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.10681676423525503


In [421]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['incont_fec'], SENSE_PA['incont_fec'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.8568943909879498


## Úlceres de pressió

In [422]:
up_AMB_PA= (AMB_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA:.2f}%")

Percentatge de úlceres de pressió: 5.88%


In [423]:
up_AMB_PA_MECVV= (AMB_PA_MECVV['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA_MECVV:.2f}%")

Percentatge de úlceres de pressió: 0.00%


In [424]:
up_SENSE_PA= (SENSE_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_SENSE_PA:.2f}%")

Percentatge de úlceres de pressió: 2.04%


#### Pvalor

##### Normalitat

In [425]:
stat, p_value = shapiro(AMB_PA['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [426]:
stat, p_value = shapiro(AMB_PA_MECVV['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [427]:
stat, p_value = shapiro(SENSE_PA['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1273
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [428]:
stat, p_value_12 = mannwhitneyu(AMB_PA['ulceras_presion'], AMB_PA_MECVV['ulceras_presion'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.49002535950179893


In [429]:
stat, p_value_13 = mannwhitneyu(AMB_PA['ulceras_presion'], SENSE_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.4440308448923743


In [430]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['ulceras_presion'], SENSE_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.6843175494727258


## Immobilitat

## Confusió

## Osteoporosi

In [431]:
ost_AMB_PA= (AMB_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge de osteoporosi: {ost_AMB_PA:.2f}%")

Percentatge de osteoporosi: 0.00%


In [432]:
ost_AMB_PA_MECVV= (AMB_PA_MECVV['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge de osteoporosi: {ost_AMB_PA_MECVV:.2f}%")

Percentatge de osteoporosi: 0.00%


In [433]:
ost_SENSE_PA= (SENSE_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge de osteoporosi: {ost_SENSE_PA:.2f}%")

Percentatge de osteoporosi: 0.00%


#### Pvalor

##### Normalitat

In [434]:
stat, p_value = shapiro(AMB_PA['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [435]:
stat, p_value_12 = mannwhitneyu(AMB_PA['osteoporosis'], AMB_PA_MECVV['osteoporosis'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 1.0


In [436]:
stat, p_value_12 = ttest_ind(AMB_PA['osteoporosis'], AMB_PA_MECVV['osteoporosis'], equal_var= False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


## Sarcopènia

In [437]:
sarc_AMB_PA= (AMB_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA:.2f}%")

Percentatge de sarcopènia: 0.00%


In [438]:
sarc_AMB_PA_MECVV= (AMB_PA_MECVV['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA_MECVV:.2f}%")

Percentatge de sarcopènia: 10.00%


In [439]:
sarc_SENSE_PA= (SENSE_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_SENSE_PA:.2f}%")

Percentatge de sarcopènia: 4.08%


#### Pvalor

##### Normalitat

In [440]:
stat, p_value = shapiro(AMB_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [441]:
stat, p_value = shapiro(AMB_PA_MECVV['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [442]:
stat, p_value = shapiro(SENSE_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2008
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [443]:
stat, p_value_12 = mannwhitneyu(AMB_PA['sarcopenia'], AMB_PA_MECVV['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.21976847950216105


In [444]:
stat, p_value_13 = mannwhitneyu(AMB_PA['sarcopenia'], SENSE_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.41519527301502746


In [445]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['sarcopenia'], SENSE_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.45734646700671056


## Trastorns del son

In [446]:
ts_AMB_PA= (AMB_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


In [447]:
ts_AMB_PA_MECVV= (AMB_PA_MECVV['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA_MECVV:.2f}%")

Percentatge de trastorns del son: 0.00%


In [448]:
ts_SENSE_PA= (SENSE_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_SENSE_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


## Dolor crònic

In [449]:
dc_AMB_PA= (AMB_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


In [450]:
dc_AMB_PA_MECVV= (AMB_PA_MECVV['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA_MECVV:.2f}%")

Percentatge de dolor crònic: 0.00%


In [451]:
dc_SENSE_PA= (SENSE_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_SENSE_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


## Iatrogènic

In [452]:
iatr_AMB_PA= (AMB_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


In [453]:
iatr_AMB_PA_MECVV= (AMB_PA_MECVV['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA_MECVV:.2f}%")

Percentatge de iatrogènic: 0.00%


In [454]:
iatr_SENSE_PA= (SENSE_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_SENSE_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


## Restrenyiment

In [455]:
est_AMB_PA= (AMB_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


In [456]:
est_AMB_PA_MECVV= (AMB_PA_MECVV['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA_MECVV:.2f}%")

Percentatge de restrenyiment: 0.00%


In [457]:
est_SENSE_PA= (SENSE_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_SENSE_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


# Historial mèdic/comorbilitats

## Malalties CV

In [458]:
cv_AMB_PA= (AMB_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA:.2f}%")

Percentatge de problemes CV: 5.88%


In [459]:
cv_AMB_PA_MECVV= (AMB_PA_MECVV['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA_MECVV:.2f}%")

Percentatge de problemes CV: 10.00%


In [460]:
cv_SENSE_PA= (SENSE_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_SENSE_PA:.2f}%")

Percentatge de problemes CV: 6.12%


#### Pvalor

##### Normalitat 

In [461]:
stat, p_value = shapiro(AMB_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [462]:
stat, p_value = shapiro(AMB_PA_MECVV['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [463]:
stat, p_value = shapiro(SENSE_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2580
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [464]:
stat, p_value_12 = mannwhitneyu(AMB_PA['CV'], AMB_PA_MECVV['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.7400102360583588


In [465]:
stat, p_value_13 = mannwhitneyu(AMB_PA['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.9858477456716862


In [466]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.6762982127217047


## Malalties del cor

In [467]:
cora_AMB_PA= (AMB_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_AMB_PA:.2f}%")

Percentatge de malalties de cor: 11.76%


In [468]:
cora_AMB_PA_MECVV= (AMB_PA_MECVV['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_AMB_PA_MECVV:.2f}%")

Percentatge de malalties de cor: 10.00%


In [469]:
cora_SENSE_PA= (SENSE_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_SENSE_PA:.2f}%")

Percentatge de malalties de cor: 14.29%


#### Pvalor

##### Normalitat

In [470]:
stat, p_value = shapiro(AMB_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3853
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [471]:
stat, p_value = shapiro(AMB_PA_MECVV['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [472]:
stat, p_value = shapiro(SENSE_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4168
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [473]:
stat, p_value_12 = mannwhitneyu(AMB_PA['probl_corazon'], AMB_PA_MECVV['probl_corazon'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.9265568396984611


In [474]:
stat, p_value_13 = mannwhitneyu(AMB_PA['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.9858477456716862


In [475]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.6762982127217047


## Malalties neurodegeneratives

In [476]:
nd_AMB_PA= (AMB_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA:.2f}%")

Percentatge de malalties neurodegeneratives: 0.00%


In [477]:
nd_AMB_PA_MECVV= (AMB_PA_MECVV['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA_MECVV:.2f}%")

Percentatge de malalties neurodegeneratives: 10.00%


In [478]:
nd_SENSE_PA= (SENSE_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_SENSE_PA:.2f}%")

Percentatge de malalties neurodegeneratives: 4.08%


#### Pvalor

##### Normalitat

In [479]:
stat, p_value = shapiro(AMB_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [480]:
stat, p_value = shapiro(AMB_PA_MECVV['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [481]:
stat, p_value = shapiro(SENSE_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2008
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [482]:
stat, p_value_12 = mannwhitneyu(AMB_PA['neurodegenerativas'], AMB_PA_MECVV['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.21976847950216105


In [483]:
stat, p_value_13 = mannwhitneyu(AMB_PA['neurodegenerativas'], SENSE_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.41519527301502746


In [484]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['neurodegenerativas'], SENSE_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.45734646700671056


## Diabetes mellitus

In [485]:
dm_AMB_PA= (AMB_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_AMB_PA:.2f}%")

Percentatge de diabetes mellitus: 23.53%


In [486]:
dm_AMB_PA_MECVV= (AMB_PA_MECVV['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_AMB_PA_MECVV:.2f}%")

Percentatge de diabetes mellitus: 10.00%


In [487]:
dm_SENSE_PA= (SENSE_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_SENSE_PA:.2f}%")

Percentatge de diabetes mellitus: 26.53%


#### Pvalor 

##### Normalitat

In [488]:
stat, p_value = shapiro(AMB_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [489]:
stat, p_value = shapiro(AMB_PA_MECVV['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [490]:
stat, p_value = shapiro(SENSE_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5514
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [491]:
stat, p_value_12 = mannwhitneyu(AMB_PA['DM'], AMB_PA_MECVV['DM'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.4120280546315189


In [492]:
stat, p_value_13 = mannwhitneyu(AMB_PA['DM'], SENSE_PA['DM'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PAN", p_value_13)

P-valor entre AMB_PA i SENSE_PAN 0.8163159339942008


In [493]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['DM'], SENSE_PA['DM'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PAN", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PAN 0.2728301710651724


## Hepatopaties

In [494]:
hp_AMB_PA= (AMB_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA:.2f}%")

Percentatge de hepatopaties: 11.76%


In [495]:
hp_AMB_PA_MECVV= (AMB_PA_MECVV['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA_MECVV:.2f}%")

Percentatge de hepatopaties: 10.00%


In [496]:
hp_SENSE_PA= (SENSE_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_SENSE_PA:.2f}%")

Percentatge de hepatopaties: 6.12%


#### Pvalor

##### Normalitat

In [497]:
stat, p_value = shapiro(AMB_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3853
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [498]:
stat, p_value = shapiro(AMB_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [499]:
stat, p_value = shapiro(SENSE_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2580
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [500]:
stat, p_value_12 = mannwhitneyu(AMB_PA['hepatopatias'], AMB_PA_MECVV['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.9265568396984611


In [501]:
stat, p_value_13 = mannwhitneyu(AMB_PA['hepatopatias'], SENSE_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.4618706673466527


In [502]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['hepatopatias'], SENSE_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.6762982127217047


## Neoplàsia

In [503]:
neopl_AMB_PA= (AMB_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_AMB_PA:.2f}%")

Percentatge de neoplàsia: 0.00%


In [504]:
neopl_AMB_PA_MECVV= (AMB_PA_MECVV['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_AMB_PA_MECVV:.2f}%")

Percentatge de neoplàsia: 0.00%


In [505]:
neopl_SENSE_PA= (SENSE_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_SENSE_PA:.2f}%")

Percentatge de neoplàsia: 6.12%


#### Pvalor

##### Normalitat

In [506]:
stat, p_value = shapiro(AMB_PA['neoplasias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [507]:
stat, p_value = shapiro(AMB_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3657
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [508]:
stat, p_value = shapiro(SENSE_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2580
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [509]:
stat, p_value_12 = mannwhitneyu(AMB_PA['neoplasias'], AMB_PA_MECVV['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 1.0


In [510]:
stat, p_value_13 = mannwhitneyu(AMB_PA['neoplasias'], SENSE_PA['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_12)

P-valor entre AMB_PA i SENSE_PA 1.0


In [511]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['neoplasias'], SENSE_PA['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.44143809205435813


## Malaltia crònica renal (creatinina > 1.5mg/dL)

Donat que aquesta columna és de tipus objecte, primer la passaré a tipus numèric per poder operar amb ella

In [512]:
AMB_PA.loc[:, 'creatinina'] = pd.to_numeric(AMB_PA['creatinina'], errors='coerce')
AMB_PA_MECVV.loc[:, 'creatinina'] = pd.to_numeric(AMB_PA_MECVV['creatinina'], errors='coerce')
SENSE_PA.loc[:, 'creatinina'] = pd.to_numeric(SENSE_PA['creatinina'], errors='coerce')

In [513]:
ecr_AMB_PA= (AMB_PA['creatinina'] >= 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [514]:
ecr_AMB_PA_MECVV= (AMB_PA_MECVV['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA_MECVV:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [515]:
ecr_SENSE_PA= (SENSE_PA['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_SENSE_PA:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 6.12%


#### Pvalor

##### Normalitat

In [516]:
stat, p_value = shapiro(AMB_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9549
Valor p = 0.7439
El nombre de files segueix una distribució normal


In [517]:
stat, p_value = shapiro(AMB_PA_MECVV['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9815
Valor p = 0.9110
El nombre de files segueix una distribució normal


In [518]:
stat, p_value = shapiro(SENSE_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7235
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [519]:
stat, p_value_12 = ttest_ind(AMB_PA['creatinina'], AMB_PA_MECVV['creatinina'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [520]:
stat, p_value_13 = mannwhitneyu(AMB_PA['creatinina'], SENSE_PA['creatinina'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [521]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['creatinina'], SENSE_PA['creatinina'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


## Fallida renal aguda (o insuficiència renal aguda)

In [522]:
ira_AMB_PA= (AMB_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA:.2f}%")

Percentatge de fallida renal aguda: 23.53%


In [523]:
ira_AMB_PA_MECVV= (AMB_PA_MECVV['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA_MECVV:.2f}%")

Percentatge de fallida renal aguda: 50.00%


In [524]:
ira_SENSE_PA= (SENSE_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_SENSE_PA:.2f}%")

Percentatge de fallida renal aguda: 26.53%


#### Pvalor

##### Normalitat

In [525]:
stat, p_value = shapiro(AMB_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [526]:
stat, p_value = shapiro(AMB_PA_MECVV['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6553
Valor p = 0.0003
El nombre de files no segueix una distribució normal.


In [527]:
stat, p_value = shapiro(SENSE_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5514
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [528]:
stat, p_value_12 = mannwhitneyu(AMB_PA['ARF'], AMB_PA_MECVV['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.17639454522781728


In [529]:
stat, p_value_13 = mannwhitneyu(AMB_PA['ARF'], SENSE_PA['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.8163159339942008


In [530]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['ARF'], SENSE_PA['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_23)

P-valor entre AMB_PA i SENSE_PA 0.14880872192595962


## Marejos/síncope/lipotímia/reacció vasovagal

In [531]:
dizsyn_AMB_PA= (AMB_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [532]:
dizsyn_AMB_PA_MECVV= (AMB_PA_MECVV['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA_MECVV:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [533]:
dizsyn_SENSE_PA= (SENSE_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_SENSE_PA:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 2.04%


#### Pvalor

##### Normalitat

In [534]:
stat, p_value = shapiro(AMB_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [535]:
stat, p_value = shapiro(AMB_PA_MECVV['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [536]:
stat, p_value = shapiro(SENSE_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1273
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [537]:
stat, p_value_12 = ttest_ind(AMB_PA['mareos'], AMB_PA_MECVV['mareos'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [538]:
stat, p_value_12 = mannwhitneyu(AMB_PA['mareos'], AMB_PA_MECVV['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 1.0


In [539]:
stat, p_value_13 = mannwhitneyu(AMB_PA['mareos'], SENSE_PA['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.5793276078637376


In [540]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['mareos'], SENSE_PA['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.6843175494727258


## VIH

In [541]:
vih_AMB_PA= (AMB_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA:.2f}%")

Percentatge de VIH: 0.00%


In [542]:
vih_AMB_PA_MECVV= (AMB_PA_MECVV['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA_MECVV:.2f}%")

Percentatge de VIH: 0.00%


In [543]:
vih_SENSE_PA= (SENSE_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_SENSE_PA:.2f}%")

Percentatge de VIH: 0.00%


#### Pvalor

##### Normalitat

In [544]:
stat, p_value = shapiro(AMB_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [545]:
stat, p_value = shapiro(AMB_PA_MECVV['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [546]:
stat, p_value = shapiro(SENSE_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


## Psicosi

In [547]:
psi_AMB_PA= (AMB_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {vih_AMB_PA:.2f}%")

Percentatge de psicosi: 0.00%


In [548]:
psi_AMB_PA_MECVV= (AMB_PA_MECVV['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_AMB_PA_MECVV:.2f}%")

Percentatge de psicosi: 0.00%


In [549]:
psi_SENSE_PA= (SENSE_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_SENSE_PA:.2f}%")

Percentatge de psicosi: 0.00%


## Deficiències nutricionals

In [550]:
nutri_AMB_PA= (AMB_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [551]:
nutri_AMB_PA_MECVV= (AMB_PA_MECVV['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA_MECVV:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [552]:
nutri_SENSE_PA= (SENSE_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_SENSE_PA:.2f}%")

Percentatge de deficiències nutricionals: 0.00%
